In [3]:
from pandas import read_csv, DataFrame
import pandas as pd
data: DataFrame = read_csv("new_class_ny_arrests.csv")

In [4]:
def identify_outliers(df):
    outliers = pd.DataFrame(columns=df.columns)
    for column in df.select_dtypes(include=['float64', 'int64']).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers_in_column = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        outliers = pd.concat([outliers, outliers_in_column])
    return outliers.drop_duplicates()

outliers = identify_outliers(data)
print(outliers)

/var/folders/sz/082127xx22j1st6q0z4tz7780000gn/T/ipykernel_12940/910998391.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  outliers = pd.concat([outliers, outliers_in_column])


         PD_CD PD_DESC  KY_CD OFNS_DESC LAW_CODE LAW_CAT_CD  ARREST_BORO  \
1205     269.0     106  121.0        30      215          1          4.0   
1408     809.0     107  364.0        38      227          0          2.0   
1446     744.0     108  359.0        34      228          0          0.0   
1481     576.0     109  234.0         7      229          0          0.0   
1501     254.0     110  351.0        30      233          0          1.0   
...        ...     ...    ...       ...      ...        ...          ...   
4545526  478.0      44  343.0        23       80          0          3.0   
4571826  567.0      21  235.0         0       25          0          1.0   
4572554  567.0      21  235.0         0       25          0          3.0   
4580131  793.0      12  118.0         8       13          1          0.0   
4650368  905.0      58  347.0        29      108          0          3.0   

        ARREST_PRECINCT  JURISDICTION_CODE  AGE_GROUP PERP_SEX  PERP_RACE  \
1205      

In [8]:
# Make a copy of the original data
data_copy = data.copy()

from sklearn.ensemble import IsolationForest
import pandas as pd

# Handle missing values by filling them with the mean of the column
data_copy.fillna(data_copy.mean(), inplace=True)

# Filter numeric columns
numeric_data = data_copy.select_dtypes(include=['float64', 'int64'])

# Fit the model
iso_forest = IsolationForest(contamination=0.05, random_state=1)
outliers = iso_forest.fit_predict(numeric_data)

# Add the outlier flag to the original data
data_copy['outlier'] = outliers

# Print the number of outliers
print(f"Number of outliers: {sum(outliers == -1)}")
print(f"Original data size: {data.shape}")
print(f"Data size without outliers: {data[data_copy['outlier'] != -1].shape}")

Number of outliers: 248435
Original data size: (4968684, 19)
Data size without outliers: (4720249, 19)
